In [1]:
from keras.applications.vgg16 import VGG16
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, multiply, BatchNormalization, ReLU, Activation
from keras.models import Model
from keras.initializers import RandomNormal

2022-11-10 22:50:26.568727: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
from datetime import datetime
import os, shutil
import copy
import pickle
import math

from config import Config
import util
from wnet import WNet
from SoftNCutLoss import NCutLoss2D

/Users/sergekeita/opt/anaconda3/envs/Bootcamp/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def main():

    config = Config()

    data_xform = transforms.Compose([
        transforms.Resize((config.input_size,config.input_size)),
        transforms.ToTensor()
    ])

    

In [4]:
config = Config()
data_xform_input = transforms.Compose([
    transforms.Resize((config.input_size,config.input_size)),
    transforms.ToTensor(),
    ])

data_xform_target = transforms.Compose([
        transforms.Resize((config.input_size,config.input_size)),
        transforms.PILToTensor(),
    ])
traindataset = torchvision.datasets.VOCSegmentation(
    root="./Data",
    year="2012",
    image_set="train",
    transform=data_xform_input,
    target_transform=data_xform_target,
    download=True)
    
traindataloader = torch.utils.data.DataLoader(traindataset,batch_size=config.batch_size,num_workers=4, shuffle=True)

#dataiter = iter(traindataloader)
#images, labels = dataiter.next()

Using downloaded and verified file: ./Data/VOCtrainval_11-May-2012.tar
Extracting ./Data/VOCtrainval_11-May-2012.tar to ./Data


In [5]:
valdataset = torchvision.datasets.VOCSegmentation(
    root="./Data",
    year="2012",
    image_set="val",
    transform=data_xform_input,
    target_transform=data_xform_target,
    download=True)
    
valdataloader = torch.utils.data.DataLoader(valdataset,batch_size=config.val_batch_size,num_workers=4, shuffle=False)

Using downloaded and verified file: ./Data/VOCtrainval_11-May-2012.tar
Extracting ./Data/VOCtrainval_11-May-2012.tar to ./Data


In [6]:

#img, labels = next(iter(traindataloader))


# Model WNET 

In [7]:
autoencoder = WNet()

In [8]:
ncutloss_layer = NCutLoss2D()

In [9]:
if torch.cuda.is_available():
    autoencoder = autoencoder.cuda

optimizerE = torch.optim.Adam(autoencoder.U_encoder.parameters(), lr=0.003)
optimizerW = torch.optim.Adam(autoencoder.parameters(), lr=0.003)

In [10]:
if config.debug:
    print(autoencoder)

WNet(
  (U_encoder): U_Net(
    (first_module): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ReLU()
      (4): Dropout(p=0.65, inplace=False)
      (5): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (7): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (8): ReLU()
      (9): Dropout(p=0.65, inplace=False)
    )
    (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (enc_modules): ModuleList(
      (0): ConvModule(
        (depthwise): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64)
        (pointwise): Conv2d(64, 128, kernel_size

### Loss criterion

In [11]:
def reconstruction_loss(x, x_prime):
        reconloss = F.mse_loss(x, x_prime, reduction='sum')
        return reconloss

### Train Loop

In [12]:
autoencoder.train()

WNet(
  (U_encoder): U_Net(
    (first_module): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ReLU()
      (4): Dropout(p=0.65, inplace=False)
      (5): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (7): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (8): ReLU()
      (9): Dropout(p=0.65, inplace=False)
    )
    (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (enc_modules): ModuleList(
      (0): ConvModule(
        (depthwise): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64)
        (pointwise): Conv2d(64, 128, kernel_size

In [1]:
progress_images, progress_expected = next(iter(valdataloader))

NameError: name 'valdataloader' is not defined

In [14]:
for epoch in range(config.num_epochs):
    running_loss = 0.0
    ncutloss = []
    reconloss = []
    for i, [inputs, outputs] in enumerate(traindataloader, 0):

        if i == 10:
            break

        if torch.cuda.is_available():
            inputs  = inputs.cuda()
            outputs = outputs.cuda()
        
        optimizerE.zero_grad()

        segmentations = autoencoder.forward_encoder(inputs)
        l_soft_n_cut     = ncutloss_layer (segmentations, inputs)
        l_soft_n_cut.backward(retain_graph=False)
        optimizerE.step()
        ncutloss.append(l_soft_n_cut)

        optimizerW.zero_grad()

        segmentations, reconstructions = autoencoder.forward(inputs)

        l_reconstruction = reconstruction_loss(
            inputs if config.variationalTranslation == 0 else outputs,
            reconstructions
        )

        reconloss.append(l_reconstruction)

        l_reconstruction.backward(retain_graph=False)  # We only need to do retain graph =true if we're backpropping from multiple heads
        optimizerW.step()

        running_loss += l_reconstruction + l_soft_n_cut #loss.item()

        if config.showSegmentationProgress and i == 0: # If first batch in epoch
            util.save_progress_image(autoencoder, progress_images, progress_expected, epoch)

    
    epoch_loss = running_loss / len(traindataloader.dataset)
    print(f"Epoch {epoch} loss: {epoch_loss:.6f}")

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


Epoch 0 loss: 2138.424561


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


Epoch 1 loss: 879.378845


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


...

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


Epoch 15 loss: 434.693054


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


Epoch 16 loss: 402.595032


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
